<a href="https://colab.research.google.com/github/GUNWOOOH/news_classification/blob/main/bert_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

코드 출처: https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

In [ ]:
from google.colab import drive
drive.mount('/content/qdrive')
!pip install konlpy
!pip install pytorch-transformers

#텍스트 분류

import os
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

path = "/content/qdrive/MyDrive/논문데이터"
file_list = os.listdir(path)
file_list2 = []

for i in file_list:
    file_list2.append(int(str(i).split('.')[0]))

df = pd.DataFrame({'날짜' : [],'신문사' : [],'제목' : [],'내용' : [],'분야' : []})

for file in file_list2:
    df_sam = pd.read_csv(path+ '/'+ str(file) +".csv", encoding='UTF8')
    df = pd.concat([df,df_sam])


import re
tit = []

df['신문사']  = np.where(df['신문사'] == 'TV조선', '조선일보', df['신문사'])
df['신문사']  = np.where(df['신문사'] == '한겨레21', '한겨레', df['신문사'])
df = df[(df['분야'] != '세계') & (df['분야'] != 'IT') & (df['분야'] != '오피니언')]
df = df[(df['신문사'] != '중앙일보') & (df['신문사'] != '경향신문')]

for i in df.제목:
    string = re.sub(r'[^ A-Za-z가-힣+]', ' ', i)
    tit.append(' '.join(string.split()))

#특정신문사 일 때 1로 만들어주기


Drive already mounted at /content/qdrive; to attempt to forcibly remount, call drive.mount("/content/qdrive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#분야 고르기
col_df = pd.DataFrame({'신문사': df.신문사,'분야': df.분야,'제목2' : tit})
col_df_sam = col_df.head(20)
#한국어 전처리 KONLPY
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
pos_tagger = Okt()
docs = [pos_tagger.pos(row) for row in col_df['제목2']] # Training Data ( 학습용 데이터 )
#docs = [pos_tagger.pos(row) for row in col_df_sam['제목2']] # Training Data ( 학습용 데이터 )
contents_tokens = []

for content in docs:
  okt_filtering = [x for x, y in content if y in ['Noun', 'Adjective', 'Verb']]
  contents_tokens.append(okt_filtering)

contents_for_vectorize = []
for content in contents_tokens:
  sentence = ''
  for word in content:
    sentence = sentence + ' ' + word
  contents_for_vectorize.append(sentence.lstrip())


In [ ]:
col_df['제목2'] = contents_for_vectorize
col_df['Y'] = list((col_df['신문사'] == '한겨레')*1)
col_df2 = col_df[col_df['분야']=='정치']
col_df2['Y'] = col_df2['Y'].astype('category')
col_df2.shape

# col_df_sam['제목2'] = contents_for_vectorize
# col_df_sam['Y'] = list((col_df_sam['신문사'] == '중앙일보')*1)
# col_df_sam2 = col_df_sam[col_df_sam['분야']=='경제']
# col_df_sam2['Y'] = col_df_sam2['Y'].astype('category')
# col_df_sam2.shape

<ipython-input-3-335ce547898d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col_df2['Y'] = col_df2['Y'].astype('category')


(115255, 4)

# 데이터 전처리

In [ ]:
from sklearn.model_selection import train_test_split

#실험코드
col_df2 = col_df[col_df['분야']=='정치']
col_df2['Y'] = col_df2['Y'].astype('category')
col_df2.shape

train_df,test_df = train_test_split(
  col_df2 , random_state=104,test_size=0.25, shuffle=True)
  # col_df_sam2 , random_state=104,test_size=0.25, shuffle=True)

<ipython-input-4-1c2570f512ae>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col_df2['Y'] = col_df2['Y'].astype('category')


In [ ]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 2]
        label = self.df.iloc[idx, 3]
        return text, label

In [ ]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [ ]:
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 5
p_itr = 200
epochs = 5
total_loss = 0
total_len = 0
total_correct = 0


model.train()
pred_all = []
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        pred_all.append(pred)

        loss.backward()
        optimizer.step()
        
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)
total_loss = 0
total_len = 0
total_correct = 0
y_pred = []
y_true = []

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)
    y_pred.extend((torch.max(torch.exp(logits), 1)[1]).data.cpu().numpy())
    y_true.extend(label.data.cpu().numpy())

print('Test accuracy: ', total_correct / total_len)

from sklearn.metrics import classification_report
 
print(classification_report(y_true, y_pred))
     

<ipython-input-8-d9a28cd834c2>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)
<ipython-input-8-d9a28cd834c2>:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)


In [ ]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)
total_loss = 0
total_len = 0
total_correct = 0
y_pred = []
y_true = []

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)
    y_pred.extend((torch.max(torch.exp(logits), 1)[1]).data.cpu().numpy())
    y_true.extend(label.data.cpu().numpy())

print('Test accuracy: ', total_correct / total_len)

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

<ipython-input-121-8fee813334e4>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)
<ipython-input-121-8fee813334e4>:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)


Test accuracy:  0.8275503122831367


In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_true, y_pred)

In [ ]:
from sklearn.metrics import classification_report
 
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91      2385
           1       0.00      0.00      0.00       497

    accuracy                           0.83      2882
   macro avg       0.41      0.50      0.45      2882
weighted avg       0.68      0.83      0.75      2882



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
(torch.max(torch.exp(logits), 1)[1])

tensor([0], device='cuda:0')

In [ ]:
torch.max(torch.exp(logits), 1)

torch.return_types.max(
values=tensor([2.1898], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([0], device='cuda:0'))

In [ ]:
torch.exp(logits)

tensor([[2.1898, 0.3519]], device='cuda:0', grad_fn=<ExpBackward0>)